AI-POWERED CUSTOMER SUPPORT CHATBOT

In [2]:
# Importing all required Libraries
import json
import spacy
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
import joblib
import sqlite3
from typing import Tuple, Dict, Any, Optional


In [3]:
# Download NLTK data
nltk.download('vader_lexicon', quiet=True)
nltk.download('punkt', quiet=True)
nltk.download('stopwords', quiet=True)

True

INTENT CLASSIFIER : This is responsible for training a Naive Bayes model to classify text inputs into predefined intent categories.


In [4]:
class IntentClassifier:
    def __init__(self, model_path: str = 'intent_classifier.pkl', vectorizer_path: str = 'count_vectorizer.pkl'):
        try:
            self.classifier = joblib.load(model_path)
            self.vectorizer = joblib.load(vectorizer_path)
        except FileNotFoundError:
            # Initialize and train if models don't exist
            self.train_classifier()

    def train_classifier(self):
        # Load and process training data
        with open('intent_data.json', 'r') as f:
            data = json.load(f)

        texts, labels = [], []
        for intent in data['intents']:
            for example in intent['examples']:
                texts.append(example)
                labels.append(intent['intent'])

        # Vectorize text
        self.vectorizer = CountVectorizer()
        X = self.vectorizer.fit_transform(texts)

        # Train classifier
        self.classifier = MultinomialNB(alpha=0.4)
        self.classifier.fit(X, labels)

        # Save models
        joblib.dump(self.classifier, 'intent_classifier.pkl')
        joblib.dump(self.vectorizer, 'count_vectorizer.pkl')

    def classify(self, text: str) -> Tuple[str, float]:
        # Vectorize input
        vector = self.vectorizer.transform([text])

        # Get prediction and confidence
        intent = self.classifier.predict(vector)[0]
        confidence = max(self.classifier.predict_proba(vector)[0])

        return intent, confidence



ENTITY EXTRACTION: This defines an entity extraction system that identifues named entities in a text using the spaCy NLP Library

In [5]:
class EntityExtractor:
    def __init__(self):
        try:
            self.nlp = spacy.load('en_core_web_sm')
        except OSError:
            spacy.cli.download('en_core_web_sm')
            self.nlp = spacy.load('en_core_web_sm')

    def extract(self, text: str) -> Dict[str, str]:
        doc = self.nlp(text)
        entities = {}

        # Extract named entities
        for ent in doc.ents:
            entities[ent.label_] = ent.text

        # Extract product keywords
        product_keywords = {
            'laptop': 'PRODUCT',
            'tablet': 'PRODUCT',
            'smartphone': 'PRODUCT',
            'smartwatch': 'PRODUCT',
            'headphone': 'PRODUCT'
        }

        text_lower = text.lower()
        for keyword, label in product_keywords.items():
            if keyword in text_lower:
                entities[label] = keyword

        return entities


DIALOGUE MANAGER : This class is responsible for managing the conversation flow ina chatbot or virtual assistant. It maintains context, updates information based on user input, and generates appropriate responses based on detected intent and confidence level

In [6]:
class DialogueManager:
    def __init__(self):
        self.context = {}
        self.last_intent = None

    def update_context(self, entities: Dict[str, str]) -> None:
        if "PRODUCT" in entities:
            self.context['product_type'] = entities["PRODUCT"]
        if "CARDINAL" in entities:
            self.context['product_model'] = entities["CARDINAL"]

    def get_response(self, intent: str, confidence: float, product_info: Optional[Dict] = None) -> str:
        if confidence < 0.4:
            return "I'm not quite sure what you're asking. Could you please rephrase that?"

        if intent == "Product Inquiry":
            if product_info:
                return f"Here's what I found about {product_info['name']}: {product_info['description']}"
            elif self.context.get('product_type'):
                return f"I'd be happy to tell you about our {self.context['product_type']}s. What specific information would you like to know?"
            else:
            # Fallback for general product inquiry
                return "We offer a range of products including laptops, smartphones, tablets, and more. Let me know if you're interested in a particular category."
        # Add more intent-specific responses
        responses = {
            "Order Status": "To check your order status, please provide your order number.",
            "Technical Support": "I can help you with technical issues. Could you describe the problem you're experiencing?",
            "Return Policy": "Our standard return period is 30 days. Would you like to know more about our return process?",
            "Product Recommendation": "I can help you find the perfect product. What features are most important to you?",
            "General Inquiry": "How can I assist you today?"
        }

        return responses.get(intent, "I'm here to help. Could you please provide more details?")


KNOWLEDGEBASE : This manages a simple SQLite database for storing and retrieving product info.

In [10]:
class KnowledgeBase:
    def __init__(self, db_path: str = "store.db"):
        self.conn = sqlite3.connect(db_path)
        self.cursor = self.conn.cursor()
        self.setup_database()

    def setup_database(self) -> None:
        self.cursor.execute('''
            CREATE TABLE IF NOT EXISTS products (
                id INTEGER PRIMARY KEY,
                name TEXT,
                category TEXT,
                description TEXT,
                price REAL
            )
        ''')
        self.conn.commit()

        # Add sample products if none exist
        self.cursor.execute('SELECT COUNT(*) FROM products')
        if self.cursor.fetchone()[0] == 0:
            self.add_sample_products()

    def add_sample_products(self) -> None:
        products = [
            ('Gaming Laptop Pro', 'laptop', 'High-performance gaming laptop with RTX 3080', 1999.99),
            ('Ultra Tablet', 'tablet', '12-inch tablet with 4K display', 799.99),
            ('SmartPhone X', 'smartphone', '5G smartphone with AI camera', 999.99)
        ]
        self.cursor.executemany(
            'INSERT INTO products (name, category, description, price) VALUES (?, ?, ?, ?)',
            products
        )
        self.conn.commit()

    def get_product_info(self, product_type: str) -> Optional[Dict[str, Any]]:
        if not product_type:
            return None

        self.cursor.execute(
            'SELECT name, description, price FROM products WHERE category = ? LIMIT 1',
            (product_type.lower(),)
        )
        result = self.cursor.fetchone()

        if result:
            return {
                'name': result[0],
                'description': result[1],
                'price': result[2]
            }
        return None


FOLLOWUP MANAGER: This class is designed to enhance user interaction in a chatbot or virtual assistant by managing follow-up questions based on the user's inquiries.

In [8]:
class FollowUpManager:
    def __init__(self):
        self.product_inquiry_followups = {
            'features': [
                "Would you like to know about specific features like battery life, processor, or storage?",
                "Should I tell you more about the technical specifications?",
                "Would you like to compare this with other similar models?",
                "Are you interested in hearing about current promotions for this product?"
            ],
            'price': [
                "Would you like to know about our financing options?",
                "Should I tell you about any ongoing discounts or deals?",
                "Would you like to know about bundle offers with accessories?",
                "Are you interested in our price match guarantee?"
            ],
            'availability': [
                "Would you like me to check stock at specific store locations?",
                "Should I notify you when this item is back in stock?",
                "Would you like to know about similar alternatives currently available?",
                "Are you interested in pre-ordering this item?"
            ]
        }

        self.technical_support_followups = {
            'troubleshooting': [
                "Have you tried restarting the device?",
                "When did you first notice this issue?",
                "Are there any error messages showing up?",
                "Has this happened before?",
                "Did you recently update your software?"
            ],
            'hardware': [
                "Is the device making any unusual sounds?",
                "Are there any visible signs of damage?",
                "How old is your device?",
                "Have you checked all the physical connections?",
                "Is the device overheating?"
            ],
            'software': [
                "Which operating system version are you using?",
                "Have you installed any new applications recently?",
                "When was the last system update?",
                "Are other applications working normally?",
                "Have you tried running in safe mode?"
            ]
        }

        self.order_status_followups = {
            'tracking': [
                "Would you like me to send you the tracking information?",
                "Should I set up delivery notifications for you?",
                "Would you like to change your delivery preferences?",
                "Should I check for any delivery delays in your area?"
            ],
            'modification': [
                "Would you like to modify your shipping address?",
                "Should I check if we can expedite your delivery?",
                "Would you like to add or remove items from your order?",
                "Are you interested in our shipping protection plan?"
            ],
            'issues': [
                "Has there been any damage to your delivered items?",
                "Would you like me to initiate a return process?",
                "Should I connect you with our shipping department?",
                "Would you like to file a claim for missing items?"
            ]
        }

        self.product_recommendation_followups = {
            'usage': [
                "What will you primarily use this device for?",
                "Do you have any specific performance requirements?",
                "What's your typical daily usage pattern?",
                "Are you looking for something portable or for desktop use?"
            ],
            'preferences': [
                "Do you have a specific budget in mind?",
                "Any particular brand preferences?",
                "What features are most important to you?",
                "Would you prefer new or refurbished products?"
            ],
            'comparison': [
                "Would you like to compare different models?",
                "Should I show you the top-rated products in this category?",
                "Would you like to know about customer reviews?",
                "Are you interested in seeing a feature comparison chart?"
            ]
        }

        self.return_policy_followups = {
            'process': [
                "Would you like me to initiate the return process now?",
                "Should I email you the return shipping label?",
                "Would you prefer to return in-store or by mail?",
                "Would you like information about our exchange policy?"
            ],
            'refund': [
                "Would you like to know about refund processing times?",
                "Should I check the status of your refund?",
                "Would you prefer store credit instead of a refund?",
                "Would you like to know about our price adjustment policy?"
            ],
            'warranty': [
                "Would you like information about extended warranty options?",
                "Should I check if your item is still under warranty?",
                "Would you like to know about our protection plans?",
                "Are you interested in our repair services?"
            ]
        }

    def get_followup(self, intent: str, subtype: str = None, context: dict = None) -> str:
        """
        Get an appropriate follow-up question based on intent and context
        """
        import random

        if intent == "Product Inquiry":
            if "price" in context.get('last_message', '').lower():
                return random.choice(self.product_inquiry_followups['price'])
            elif "available" in context.get('last_message', '').lower():
                return random.choice(self.product_inquiry_followups['availability'])
            return random.choice(self.product_inquiry_followups['features'])

        elif intent == "Technical Support":
            if any(word in context.get('last_message', '').lower() for word in ['crash', 'error', 'not working']):
                return random.choice(self.technical_support_followups['software'])
            elif any(word in context.get('last_message', '').lower() for word in ['hot', 'sound', 'broken']):
                return random.choice(self.technical_support_followups['hardware'])
            return random.choice(self.technical_support_followups['troubleshooting'])

        elif intent == "Order Status":
            if "where" in context.get('last_message', '').lower():
                return random.choice(self.order_status_followups['tracking'])
            elif "wrong" in context.get('last_message', '').lower():
                return random.choice(self.order_status_followups['issues'])
            return random.choice(self.order_status_followups['modification'])

        elif intent == "Product Recommendation":
            if "compare" in context.get('last_message', '').lower():
                return random.choice(self.product_recommendation_followups['comparison'])
            elif "budget" in context.get('last_message', '').lower():
                return random.choice(self.product_recommendation_followups['preferences'])
            return random.choice(self.product_recommendation_followups['usage'])

        elif intent == "Return Policy":
            if "refund" in context.get('last_message', '').lower():
                return random.choice(self.return_policy_followups['refund'])
            elif "warranty" in context.get('last_message', '').lower():
                return random.choice(self.return_policy_followups['warranty'])
            return random.choice(self.return_policy_followups['process'])

        # Default fallback
        return "Is there anything specific you'd like to know more about?"

    def generate_response_with_followup(self, intent: str, base_response: str, context: dict) -> str:
        """
        Combine the base response with an appropriate follow-up question
        """
        followup = self.get_followup(intent, context=context)

        # Add connecting phrase based on the type of response
        if "?" in base_response:
            connector = " Also, "
        else:
            connector = " "

        return f"{base_response}{connector}{followup}"


CHATBOT : This is a final AI Customer Support chatbot designed to process user inputs, classify intents , extract entities , analyze sentiment, manage dialogue, retrive product information and generate responses.

In [13]:
class Chatbot:
    def __init__(self):
        self.intent_classifier = IntentClassifier()
        self.entity_extractor = EntityExtractor()
        self.dialogue_manager = DialogueManager()
        self.knowledge_base = KnowledgeBase()
        self.sentiment_analyzer = SentimentIntensityAnalyzer()
        self.followup_manager = FollowUpManager()
        self.conversation_history = []

    def process_input(self, user_input: str) -> str:
        try:
            # Store user input in conversation history
            self.conversation_history.append({"role": "user", "message": user_input})

            # Classify intent and get confidence score
            intent, confidence = self.intent_classifier.classify(user_input)

            # Extract entities from user input
            entities = self.entity_extractor.extract(user_input)

            # Analyze sentiment
            sentiment_scores = self.sentiment_analyzer.polarity_scores(user_input)

            # Update dialogue context
            self.dialogue_manager.context.update({
                "last_message": user_input,
                "intent": intent,
                "confidence": confidence,
                "sentiment": sentiment_scores,
                "entities": entities
            })

            # Get product information if available
            product_info = None
            if 'PRODUCT' in entities:
                product_info = self.knowledge_base.get_product_info(entities['PRODUCT'])
                self.dialogue_manager.context['product_info'] = product_info

            # Generate base response
            base_response = self.dialogue_manager.get_response(intent, confidence, product_info)

            # Adjust response based on sentiment
            if sentiment_scores['compound'] <= -0.05:
                base_response = "I understand your concern. " + base_response
            elif sentiment_scores['compound'] >= 0.05:
                base_response = "I'm glad to help! " + base_response

            # Generate full response with follow-up
            if confidence >= 0.6:  # Only add follow-up if confidence is high enough
                full_response = self.followup_manager.generate_response_with_followup(
                    intent,
                    base_response,
                    self.dialogue_manager.context
                )
            else:
                full_response = base_response + " Could you please provide more details about what you're looking for?"

            # Store bot response in conversation history
            self.conversation_history.append({"role": "bot", "message": full_response})

            return full_response

        except Exception as e:
            print(f"Error processing input: {str(e)}")
            return "I apologize, but I'm having trouble processing your request. Could you please try again?"

def main():
    print("Initializing chatbot...")
    chatbot = Chatbot()
    print("Welcome to AI Customer Support! Type 'exit' to end the conversation.")

    while True:
        try:
            user_input = input("\nYou: ").strip()

            if user_input.lower() in ['exit', 'quit', 'bye','thankyou']:
                print("Thank you for chatting! Goodbye!")
                break

            if not user_input:
                print("Bot: Please type something so I can help you.")
                continue

            response = chatbot.process_input(user_input)
            print(f"Bot: {response}")

        except KeyboardInterrupt:
            print("\nBot: Goodbye! Have a great day!")
            break
        except Exception as e:
            print(f"Bot: I encountered an error. Please try again. Error: {str(e)}")

if __name__ == "__main__":
    main()

Initializing chatbot...
Welcome to AI Customer Support! Type 'exit' to end the conversation.

You: Bye
Thank you for chatting! Goodbye!
